In [ ]:
#Autor:Andrés
#Descripción: se trata de un primera aproximación de un estudio de cada variable con la variable respuesta
#Estado: no acabado

In [22]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns

ruta = os.path.join(os.getcwd(), "..", "..","data", "processed", "peleas_ponderadas.parquet")
df = pd.read_parquet(ruta)

In [23]:
from scipy.stats import pointbiserialr




correlations = {}
for col in df.drop(columns=['WINNER',"Peleador_A", "Peleador_B", "DATE"]).columns:  # Suponiendo que la variable de salida se llama "target"
    corr, p_value = pointbiserialr(df[col], df['WINNER'])
    correlations[col] = (corr, p_value)

# Convertir a un DataFrame para mejor visualización
import pandas as pd
correlation_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlación', 'p-valor'])
print(correlation_df)


              Correlación       p-valor
KD_A            -0.024382  1.506658e-01
KD_B             0.012974  4.444807e-01
SIG_STR_A       -0.063575  1.763727e-04
SIG_STR_B        0.036441  3.168031e-02
TD_PORC_A       -0.062347  2.349634e-04
...                   ...           ...
SIG_STR_DIFF    -0.075540  8.255239e-06
TD_DIFF         -0.070008  3.610295e-05
SUB_ATT_DIFF    -0.060470  3.609256e-04
REV_DIFF         0.013753  4.175913e-01
CTRL_DIFF       -0.098729  5.444442e-09

[74 rows x 2 columns]


In [24]:
correlation_df.sort_values("Correlación", ascending=False)

,Correlación,p-valor
Derrotas_Decision_A,0.112335,3.101318e-11
Derrotas_KO_A,0.110467,6.554597e-11
Peleas_A,0.102998,1.153715e-09
STR_HEAD_B_x,0.082380,1.153578e-06
Victorias_Decision_A,0.071204,2.647195e-05
...,...,...
SUB_ATT_A,-0.088419,1.776358e-07
STR_GROUND_A_x,-0.096848,1.056940e-08
CTRL_DIFF,-0.098729,5.444442e-09
CTRL_A,-0.102952,1.173734e-09


In [25]:
from scipy.stats import ttest_ind

ttest_results = {}
for col in df.drop(columns=['WINNER',"Peleador_A", "Peleador_B", "DATE"]).columns:
    group_0 = df[df['WINNER'] == 0][col]
    group_1 = df[df['WINNER'] == 1][col]
    t_stat, p_value = ttest_ind(group_0, group_1)
    ttest_results[col] = (t_stat, p_value)

ttest_df = pd.DataFrame.from_dict(ttest_results, orient='index', columns=['t-stat', 'p-valor'])

ttest_df.sort_values("t-stat", ascending=False)

,t-stat,p-valor
STR_GROUND_A_y,6.445761,1.308756e-10
CTRL_A,6.100473,1.173734e-09
CTRL_DIFF,5.847719,5.444442e-09
STR_GROUND_A_x,5.735247,1.056940e-08
SUB_ATT_A,5.231975,1.776358e-07
...,...,...
Victorias_Decision_A,-4.207518,2.647195e-05
STR_HEAD_B_x,-4.872091,1.153578e-06
Peleas_A,-6.103251,1.153715e-09
Derrotas_KO_A,-6.551078,6.554597e-11


In [26]:
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['WINNER', "DATE", "Peleador_A", "Peleador_B"])
y = df['WINNER']

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

importances = pd.DataFrame({'Variable': X.columns, 'Importancia': rf.feature_importances_})
importances = importances.sort_values(by='Importancia', ascending=False)
print(importances)


          Variable  Importancia
53        Puntos_B     0.021719
21    STR_HEAD_B_x     0.019673
69    SIG_STR_DIFF     0.018932
42  STR_GROUND_A_y     0.018388
70         TD_DIFF     0.018085
..             ...          ...
63   Derrotas_KO_B     0.006729
8            REV_A     0.005894
64  Derrotas_Sub_A     0.005846
9            REV_B     0.005488
65  Derrotas_Sub_B     0.004841

[74 rows x 2 columns]
